# Pipeline Example

In [ ]:
from sudareph import *

import logging

logging.basicConfig(level=logging.DEBUG)

## Module Definition

In [ ]:
@work_cls('Sum')
class Summation:
    def __init__(self, prefix: str):
        self.prefix = prefix

    def __call__(self, str1: str) -> str:
        return self.prefix + str1

@work_fn('sum')
def sum(a):
    return 'ASDF:' + a

# Level1: Use Module as function

In [ ]:
work = sum
result = work('INPUT')

print(result.output)

work = Summation('ASDF:')
result = work('Input')

print(result.output)

## Level2: Use Module as smart function

In [ ]:
input = Data('name', 'value')
work = Summation('POST:')

result = input > work

print(result.output)

## Level3: Component can be joint

In [ ]:
pipeline = Summation('PRE:') >> Summation('POST:')
input = Data('name', 'value')

result = input > pipeline

print(result.output)

(Data('name', 'value') > Summation('PRE:') >> Summation('POST:')).output

## Parallel Workflow

In [ ]:
pipeline = Summation('Outer:') >> Parallel(
    'Parallel',
    A=Summation('Inner1') >> Summation('Inner2'),
    B=Summation('Inner1-1') >> Summation('Inner2-1'),
)

(Data('name', 'ARG') > pipeline).output


pipeline = Summation('Outer:') >> Parallel(
    'Parallel',
    A=Parallel(
        'Parallel',
        A=Summation('Inner1') >> Summation('Inner2'),
        B=Summation('Inner1-1') >> Summation('Inner2-1'),
    ),
    B=Summation('Inner1-1') >> Summation('Inner2-1'),
)

(Data('name', 'ARG') > pipeline).output